# Libraries

In [1]:
import numpy as np
import pandas as pd

# Data import

In [4]:
data = pd.read_csv("./data/wina.pl_raw.csv")

# Data cleaning

#### Basic informations about columns

In [9]:
data[sorted(data.columns)].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2251 entries, 0 to 2250
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   alkohol zawartość       3 non-null      object 
 1   apelacja                1650 non-null   object 
 2   bio                     675 non-null    object 
 3   bio2                    129 non-null    object 
 4   bio3                    34 non-null     object 
 5   czy winnica bio         520 non-null    object 
 6   falstaff                84 non-null     object 
 7   grona                   2197 non-null   object 
 8   grona10                 1 non-null      object 
 9   grona11                 1 non-null      object 
 10  grona12                 1 non-null      object 
 11  grona13                 1 non-null      object 
 12  grona2                  607 non-null    object 
 13  grona3                  331 non-null    object 
 14  grona4                  94 non-null     

#### Rejecting products which is not wines and unnecessary columns

In [17]:
# removing rows
data = data[~(data['name'] == 'Chateau ISH Sparkling Rose w puszce (bezalkoholowe)')]
data = data[~(data['name'] == 'Chateau ISH Sparkling White w puszce (bezalkoholowe)')]
data = data[data['ilość win'].isnull()]
data.reset_index(inplace=True, drop=True)

# removing columns
data.drop(['ilość win', 'wino koszerne', 'producent', 'wysyłka w ciągu', 'temperatura podawania2'], axis=1, inplace=True)

#### Name

In [21]:
def isContainNumber(text: str) -> str:
    '''
    check there is a number in the text
    '''
    for digit in [str(i) for i in range(10)]:
        if digit in list(text):
            return True
    return False

namesContainingNumbers = []
for i in range(len(data)):
    for word in data.loc[i, 'name'].split(' '):
        if isContainNumber(word):
            namesContainingNumbers.append(i)
            break

In [22]:
# two names in dataset contains vintage of wine that was not included
print(f"{data.loc[172, 'rocznik']}\t{data.loc[172, 'name']}")
print(f"{data.loc[173, 'rocznik']}\t{data.loc[173, 'name']}")

data.loc[172, 'rocznik'] = 2018
data.loc[173, 'rocznik'] = 2018

nan	Barone Pizzini Naturae 2018 Franciacorta DOCG
nan	Barone Pizzini Rosé 2018 Franciacorta DOCG


In [23]:
units = ['ml', 'Ml', 'ML', 'l', 'L', 'litrów', 'NV']
digits = [str(i) for i in range(10)]

# removing vintage and volume from wine names
for i in namesContainingNumbers:
    words = data.loc[i, 'name'].split(' ')
    newName = ''
    for j in range(len(words)):
        isAdd = True

        # if contain number
        if isContainNumber(words[j]):
            # if it is date
            if len(list(words[j])) == 4:
                isAdd = False
            
            # if it is volume
            if j != len(words)-1:
                if words[j+1] in units:
                    isAdd = False

            # if it is volume with unit
            if words[j][-1:] in units[:3] or words[j][-2:] in units[3:5]:
                isAdd = False

        # if it is unit
        if words[j] in units:
            isAdd = False
        
        if isAdd:
            newName += words[j] + ' '

    data.loc[i, 'name'] = newName[:-1]

#### Price

In [24]:
# convert price into number data type
data['price (PLN)'] = data['price'].transform(lambda text: text.replace(u'\xa0', '')[:-3].replace(',', '.')).astype('float32')
data.drop(['price'], axis=1, inplace=True)

#### Region

In [25]:
cols = ['region', 'region2']
regions = []

# connect two columns into one
for row in range(len(data)):
    region = ''
    for value in data.loc[row, cols]:
        if pd.isnull(value):
            break

        region += value + ', '

    if region == '':
        regions.append(np.nan)
    else:
        regions.append(region[:-2])
    
data['regionConnected'] = pd.DataFrame({'regionConnected': regions})
data.drop(cols, axis=1, inplace=True)
data.rename(columns={'regionConnected': 'region'}, inplace=True)

#### Vintage

In [26]:
# replace NV value into NAN
data['rocznik'] = data['rocznik'].replace('NV', np.nan)

# replace vintage between column
index = data[(data['rocznik'].isnull()) & (data['rocznik2'].notnull())].index[0]
data.loc[index, 'rocznik'] = data.loc[index, 'rocznik2']

data.drop(['rocznik2', 'rocznik3'], axis=1, inplace=True)
data.rename(columns={'rocznik': 'vintage'}, inplace=True)
data['vintage'] = data['vintage'].astype('float32')

#### Volume

In [27]:
for i in data[data['objętość'].notnull()].index:
    data.loc[i, 'pojemność'] = data.loc[i, 'objętość']

data.drop(['objętość'], axis=1, inplace=True)

def resetVolume(volume):
    '''
    convert volume to number data type
    '''
    if pd.isnull(volume):
        return np.nan

    unit = volume.split(' ')[1]
    volume = volume.split(' ')[0].replace(',', '.')

    if unit == 'ml':
        return float(volume) / 1000
    else:
        return float(volume)

data['volume (liters)'] = data['pojemność'].transform(resetVolume).astype('float32')
data.drop(['pojemność'], axis=1, inplace=True)

#### Alcohol

In [28]:
# replace values from alkohol zawartość column into zawartość alkoholu
for i in data[data['alkohol zawartość'].notnull()].index:
    data.loc[i, 'zawartość alkoholu'] = data.loc[i, 'alkohol zawartość']

data.drop(['alkohol zawartość', 'zawartość alkoholu2'], axis=1, inplace=True)

def resetAlcohol(alco):
    '''
    convert alcohol value to number data type
    '''
    if pd.isnull(alco):
        return np.nan
    return alco.replace(',', '.')[:-1]

data['alcohol (%)'] = data['zawartość alkoholu'].transform(resetAlcohol).astype('float32')
data.drop(['zawartość alkoholu'], axis=1, inplace=True)

#### Serving temperature

In [29]:
def resetTemperature(temp):
    '''
    removing C sign from serving temperature (C) column
    '''
    if pd.isnull(temp):
        return np.nan
    return temp.replace(' st.C', '')

data['serving temperature (C)'] = data['temperatura podawania'].transform(resetTemperature)
data.drop(['temperatura podawania'], axis=1, inplace=True)

#### Color and kind

In [30]:
color = ['białe', 'czerwone', 'różowe', 'pomarańczowe']
kind = ['szampan', 'musujące', 'sherry', 'porto']

kolor = []
rodzaj = []

for rodzaj1, rodzaj2 in zip(data['rodzaj wina'], data['rodzaj wina2']):
    if rodzaj1 in color:
        kolor.append(rodzaj1)
    elif rodzaj2 in color:
        kolor.append(rodzaj2)
    else:
        kolor.append(np.nan)

    if rodzaj1 in kind:
        rodzaj.append(rodzaj1)
    elif rodzaj2 in kind:
        rodzaj.append(rodzaj2)
    else:
        rodzaj.append(np.nan)

data[['color', 'kind']] = pd.DataFrame({
    'color': kolor,
    'kind': rodzaj
})
data.drop(['rodzaj wina', 'rodzaj wina2'], axis=1, inplace=True)

#### Medals

In [31]:
cols = ['medale', 'medale2', 'medale3']

medals = []

for row in range(len(data)):
    medal = ''
    for value in data.loc[row, cols]:
        if pd.isnull(value):
            break

        medal += value + ', '

    if medal == '':
        medals.append(np.nan)
    else:
        medals.append(medal[:-2])
    
data['medals'] = pd.DataFrame({'medals': medals})
data.drop(cols, axis=1, inplace=True)

#### Wegan and Natural

In [32]:
wegan = []
natural = []

for bio, bio2, bio3, isBio, isWegan in zip(data['bio'], data['bio2'], data['bio3'], data['czy winnica bio'], data['wino wegańskie']):
    if bio == 'weganskie' or \
       bio2 == 'weganskie' or \
       bio3 == 'weganskie' or \
       isWegan == 'tak':
        wegan.append(True)
    else:
        wegan.append(False)

    if (bio in ['organiczne', 'naturalne']) or \
       (bio2 in ['organiczne', 'naturalne']) or \
       (bio3 in ['organiczne', 'naturalne']) or \
       isBio == 'tak – uprawa ograniczna/naturalna':
        natural.append(True)
    else:
        natural.append(False)

bio = pd.DataFrame({
    'wegan': wegan,
    'natural': natural
})

data = pd.concat([data, bio], axis=1)
data.drop(['bio', 'bio2', 'bio3', 'czy winnica bio', 'wino wegańskie'], axis=1, inplace=True)

#### Punctation

In [33]:
punctation = []

for i in range(len(data)):
    points = []

    if not pd.isnull(data.loc[i, 'guia penin']):
        points.append(int(data.loc[i, 'guia penin'][:2]))

    if not pd.isnull(data.loc[i, 'falstaff']):
        points.append(int(data.loc[i, 'falstaff'][-3:]))

    if not pd.isnull(data.loc[i, 'james suckling']):
        points.append(int(data.loc[i, 'james suckling'][:2]))

    if not pd.isnull(data.loc[i, 'oceny prasy']):
        points.append(int(data.loc[i, 'oceny prasy'][-3:]))

    if not pd.isnull(data.loc[i, 'wine spectator']):
        points.append(int(data.loc[i, 'wine spectator'][:2]))

    if len(points) > 0:
        punctation.append(np.mean(points))
    else:
        punctation.append(np.nan)

data['punctation'] = pd.Series(punctation).astype('float32')
data.drop(['guia penin', 'falstaff', 'james suckling', 'oceny prasy', 'wine spectator'], axis=1, inplace=True)

#### Grapes

In [34]:
cols = ['grona'] + [f'grona{i}' for i in range(2, 14)]

grapes = []

for row in range(len(data)):
    grape = ''
    for value in data.loc[row, cols]:
        if pd.isnull(value):
            break

        grape += value + ', '

    if grape == '':
        grapes.append(np.nan)
    else:
        grapes.append(grape[:-2])
    
data['grapes'] = pd.DataFrame({'grapes': grapes})
data.drop(cols, axis=1, inplace=True)

#### Appellation

In [35]:
data['apelacja'].replace('- brak apelacji', np.nan, inplace=True)

#### Translating column names and changing order of columns

In [36]:
# translation
data = data.rename(columns={
    'kraj': 'country',
    'apelacja': 'appellation',
    'winnica': 'vineyard',
    'smak': 'taste',
    'styl': 'style'
})

# change order
data = data[[
    'name',
    'link',
    'price (PLN)',
    'country',
    'region',
    'appellation',
    'vineyard',
    'vintage',
    'volume (liters)',
    'alcohol (%)',
    'serving temperature (C)',
    'color',
    'kind',
    'taste',
    'style',
    'medals',
    'wegan',
    'natural',
    'punctation',
    'grapes'
]]

data.to_csv('wina.pl_clean.csv', index=False)